This notebook demonstrates integration of functionality from SNT with Python through pyimagej.  
The context is the computation and rendering of the convex hull enclosing all axonal/dendritic terminal nodes.    

In [1]:
import sys
#!conda install --yes --prefix {sys.prefix} -c conda-forge pyimagej openjdk=8 scipy

In [2]:
import imagej
import numpy as np
from scipy.spatial import ConvexHull
from collections import defaultdict

In [3]:
# Initialize Fiji with GUI support.
ij = imagej.init(r'C:\Users\cam\Desktop\Fiji.app', headless=False)

from jnius import autoclass, cast

In [4]:
# Import relevant SNT (Java) classes.
HashSet = autoclass('java.util.HashSet')
PointInImage = autoclass('sc.fiji.snt.util.PointInImage')
MouseLightLoader = autoclass('sc.fiji.snt.io.MouseLightLoader')
Tree = autoclass('sc.fiji.snt.Tree')
TreeAnalyzer = autoclass('sc.fiji.snt.analysis.TreeAnalyzer')
Viewer = autoclass('sc.fiji.snt.viewer.Viewer3D')

In [5]:
def convex_hull(tree):
    """Computes the convex hull of the input Tree
    and return a Java ArrayList containing the vertices 
    representing the hull.

    Returns:
        verts_java (ArrayList): The list containing the hull vertices.
    Args:
        tree (Tree): The SNT Tree object."""
    
    tips = TreeAnalyzer(tree).getTips()
    compartment_dict = defaultdict(list)
    for t in list(tips.toArray()):
        annotation = t.getAnnotation()
        if annotation is not None:
            compartment_dict[annotation.name()].append(t)
    
    max_compartment = max(compartment_dict, key= lambda x: len(compartment_dict[x]))
    print(max_compartment)
    
    compartment_tips = compartment_dict[max_compartment]
    return ij.py.to_java(compartment_tips)

In [6]:
def run():
    
    # Fetch swc from MouseLight database by ID.
    loader = MouseLightLoader('AA1044')
    if not loader.isDatabaseAvailable():
        print("Could not connect to ML database", "Error")
        return
    if not loader.idExists():
        print("Somewhow the specified id was not found", "Error")
        return

    tree_axon = loader.getTree("axon")
    
    #verts_axon = convex_hull(tree_axon)
    #verts_axon = TreeAnalyzer(tree_axon).getTips()

    # Visualize the result using SNT Viewer3D.
    viewer = Viewer()
    viewer.add(tree_axon)
    axon_hull = viewer.annotateSurface(tree_axon.getNodes(), "Axon Hull")
    axon_hull.setColor("red")
    viewer.show()

In [7]:
run()

JavaException: JVM exception occurred: com/github/quickhull3d/Point3d